## POS tagging using modified Viterbi

### Data Preparation

In [3]:
#Importing libraries
import nltk
import re
import pprint,time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import requests
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from collections import Counter

In [4]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[:1])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')]]


In [5]:
# Splitting data into train and test
random.seed(1000)
train_set, val_set = train_test_split(nltk_data,test_size=0.05)
print(len(train_set))
print(len(val_set))

3718
196


In [6]:
train_tagged_words=[tup for sent in train_set for tup in sent]
len(train_tagged_words)

95638

In [7]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
V=set(tokens)


In [8]:
# number of tags
tags=[pair[1] for pair in train_tagged_words]
T = set(tags)
print(len(T))
print(T)

12
{'DET', 'VERB', 'X', 'PRON', 'CONJ', 'PRT', '.', 'ADJ', 'NOUN', 'ADV', 'NUM', 'ADP'}


### Build the vanilla Viterbi based POS tagger

In [9]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))


In [10]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):

    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)


In [12]:
#Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [13]:
# examples
print(t2_given_t1(t2='NOUN', t1='ADJ'))


(4262, 6078)


In [14]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
        

In [15]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,DET,VERB,X,PRON,CONJ,PRT,.,ADJ,NOUN,ADV,NUM,ADP
DET,0.005410,0.039793,0.045804,0.003486,0.000481,0.000240,0.017793,0.204616,0.638134,0.012743,0.022121,0.009377
VERB,0.135204,0.168540,0.218234,0.035972,0.005117,0.031553,0.034809,0.066052,0.109853,0.081014,0.022405,0.091247
X,0.055062,0.205173,0.074738,0.055379,0.010632,0.183910,0.163440,0.016661,0.061726,0.026341,0.002698,0.144240
PRON,0.009560,0.483365,0.092161,0.008031,0.005354,0.013002,0.040918,0.074952,0.208031,0.034034,0.007266,0.023327
CONJ,0.122077,0.157156,0.008419,0.058466,0.000468,0.004677,0.034612,0.117867,0.344715,0.056127,0.041628,0.053789
PRT,0.101142,0.401631,0.013051,0.018597,0.002284,0.001958,0.041436,0.084829,0.246982,0.010114,0.057096,0.020881
.,0.174383,0.088453,0.027112,0.066294,0.057827,0.002252,0.093317,0.044226,0.220141,0.053594,0.080886,0.091425
ADJ,0.004936,0.011681,0.020895,0.000494,0.016453,0.010201,0.064988,0.066798,0.701218,0.004607,0.021224,0.076505
NOUN,0.013160,0.147536,0.029025,0.004752,0.042477,0.044195,0.239801,0.011807,0.263525,0.016998,0.009285,0.177438
ADV,0.068988,0.346932,0.022886,0.014594,0.006965,0.014262,0.136982,0.126036,0.032504,0.079270,0.030514,0.120066


In [16]:
tags_df.loc['.', :]

DET     0.174383
VERB    0.088453
X       0.027112
PRON    0.066294
CONJ    0.057827
PRT     0.002252
.       0.093317
ADJ     0.044226
NOUN    0.220141
ADV     0.053594
NUM     0.080886
ADP     0.091425
Name: ., dtype: float32

In [29]:
# Viterbi Heuristic

no_tag=[]
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
           
        #state probabilities and hence pmax will be 0 whenever a new word is encountered        
        if pmax == 0 :
            no_tag.append(words[key])
         
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



In [30]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1000)

# choose random 5 sents
rndom = [random.randint(1,len(val_set)) for x in range(5)]

# list of sents
test_run = [val_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('In', 'ADP'),
  ('major', 'ADJ'),
  ('market', 'NOUN'),
  ('activity', 'NOUN'),
  (':', '.')],
 [('On', 'ADP'),
  ('London', 'NOUN'),
  ("'s", 'PRT'),
  ('Stock', 'NOUN'),
  ('Exchange', 'NOUN'),
  (',', '.'),
  ('Reuters', 'NOUN'),
  ('shares', 'NOUN'),
  ('rose', 'VERB'),
  ('five', 'NUM'),
  ('pence', 'NOUN'),
  ('to', 'PRT'),
  ('913', 'NUM'),
  ('pence', 'NOUN'),
  ('-LRB-', '.'),
  ('$', '.'),
  ('14.43', 'NUM'),
  ('*U*', 'X'),
  ('-RRB-', '.'),
  ('.', '.')],
 [('The', 'DET'),
  ('court', 'NOUN'),
  ('hearing', 'NOUN'),
  ('began', 'VERB'),
  ('in', 'ADP'),
  ('early', 'ADJ'),
  ('October', 'NOUN'),
  ('at', 'ADP'),
  ('the', 'DET'),
  ('request', 'NOUN'),
  ('of', 'ADP'),
  ('Anthony', 'NOUN'),
  ('Hazell', 'NOUN'),
  (',', '.'),
  ('district', 'NOUN'),
  ('auditor', 'NOUN'),
  ('for', 'ADP'),
  ('Hammersmith', 'NOUN'),
  (',', '.'),
  ('who', 'PRON'),
  ('*T*-63', 'X'),
  ('argued', 'VERB'),
  ('that', 'ADP'),
  ('local', 'ADJ'),
  ('councils', 'NOUN'),
  ('are', 'VERB'),


In [31]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [32]:
print("Time taken in seconds: ", difference)
print(tagged_seq[0:30])


Time taken in seconds:  26.10124897956848
[('In', 'ADP'), ('major', 'ADJ'), ('market', 'NOUN'), ('activity', 'NOUN'), (':', '.'), ('On', 'ADP'), ('London', 'NOUN'), ("'s", 'PRT'), ('Stock', 'NOUN'), ('Exchange', 'NOUN'), (',', '.'), ('Reuters', 'NOUN'), ('shares', 'NOUN'), ('rose', 'VERB'), ('five', 'NUM'), ('pence', 'NOUN'), ('to', 'PRT'), ('913', 'DET'), ('pence', 'NOUN'), ('-LRB-', '.'), ('$', '.'), ('14.43', 'DET'), ('*U*', 'X'), ('-RRB-', '.'), ('.', '.'), ('The', 'DET'), ('court', 'NOUN'), ('hearing', 'NOUN'), ('began', 'VERB'), ('in', 'ADP')]


In [33]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy_vanilla_Viterbi = len(check)/len(tagged_seq)
accuracy_vanilla_Viterbi

print("Vanilla Viterbi Algorithm's Accuracy :",accuracy_vanilla_Viterbi)

Vanilla Viterbi Algorithm's Accuracy : 0.9202898550724637


In [34]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [35]:
incorrect_tagged_cases

[[('to', 'PRT'), (('913', 'DET'), ('913', 'NUM'))],
 [('$', '.'), (('14.43', 'DET'), ('14.43', 'NUM'))],
 [('Anthony', 'NOUN'), (('Hazell', 'DET'), ('Hazell', 'NOUN'))],
 [('district', 'NOUN'), (('auditor', 'DET'), ('auditor', 'NOUN'))],
 [("n't", 'ADV'), (('vested', 'DET'), ('vested', 'VERB'))],
 [('Industry', 'NOUN'), (('summoned', 'DET'), ('summoned', 'VERB'))],
 [('.', '.'), (('Cartoonist', 'DET'), ('Cartoonist', 'NOUN'))],
 [('Cartoonist', 'NOUN'), (('Garry', 'DET'), ('Garry', 'NOUN'))],
 [('and', 'CONJ'), (('punish', 'DET'), ('punish', 'VERB'))],
 [('a', 'DET'), (('screenwriters', 'DET'), ('screenwriters', 'NOUN'))],
 [('screenwriters', 'NOUN'), (("'", '.'), ("'", 'PRT'))]]

In [37]:
##Entries which were not tagged
print(no_tag)

['913', '14.43', 'Hazell', 'auditor', 'vested', 'summoned', 'Cartoonist', 'Garry', 'punish', 'screenwriters']


In [38]:
## Testing Vanilla Viterbi Algorithm on Test Sentences
sentence_test = 'Twitter is the best networking social site.Man is a social animal. Data science is an emerging field.Data science jobs are high in demand.'
words = word_tokenize(sentence_test)
start = time.time()
tagged_seq = Viterbi(words)
end = time.time()
difference = end-start

In [39]:
print(tagged_seq)
print(difference)

[('Twitter', 'DET'), ('is', 'VERB'), ('the', 'DET'), ('best', 'ADJ'), ('networking', 'NOUN'), ('social', 'ADJ'), ('site.Man', 'DET'), ('is', 'VERB'), ('a', 'DET'), ('social', 'ADJ'), ('animal', 'DET'), ('.', '.'), ('Data', 'NOUN'), ('science', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('emerging', 'VERB'), ('field.Data', 'DET'), ('science', 'NOUN'), ('jobs', 'NOUN'), ('are', 'VERB'), ('high', 'ADJ'), ('in', 'ADP'), ('demand', 'NOUN'), ('.', '.')]
4.647507190704346


### Solve the problem of unknown words

In [40]:
#approach 1

### Printing the unique tag and counter 
tagged_words = [tup for sent in nltk_data for tup in sent]
tags = [pair[1] for pair in tagged_words]
tagged_counts=Counter(tags)
tagged_counts

Counter({'.': 11715,
         'ADJ': 6397,
         'ADP': 9857,
         'ADV': 3171,
         'CONJ': 2265,
         'DET': 8725,
         'NOUN': 28867,
         'NUM': 3546,
         'PRON': 2737,
         'PRT': 3219,
         'VERB': 13564,
         'X': 6613})

In [41]:
###  Lexicon and Rule Based Tagging
# Lexicon (or unigram tagger)
unigram_tagger = nltk.UnigramTagger(train_set)
unigram_tagger.evaluate(val_set)

0.8987693529178246

In [50]:
# specify patterns for tagging
# example from the NLTK book
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*ould$', 'VERB'),              # modals
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A_Z].*','NOUN'),
    (r'.*', 'NOUN')                    # nouns
]

regexp_tagger = nltk.RegexpTagger(patterns)
# help(regexp_tagger)
regexp_tagger.evaluate(val_set)

0.3445811830091306

In [51]:
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

# lexicon backed up by the rule-based tagger
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)

print("Lexicon and Rule Based Tagging Algorithm's Accuracy :",lexicon_tagger.evaluate(val_set))


Lexicon and Rule Based Tagging Algorithm's Accuracy : 0.9491861849940453


In [52]:
def tag_unknown_word(word):
    
    # fetch tag based on the lexicon & rule based tagger
    res = [val[1] for val in rule_based_tagger.tag([word])]
    
    # return result
    return(str(res[0]))

In [64]:
print(tag_unknow_word('Google'))

NOUN


In [54]:
# Modified Viterbi Heuristic
def Modified_Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # If pmax is zero that means it is a UNKNOWN WORD

        if pmax == 0 :            
                state_max = tag_unknow_word(words[key])               
        else :
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)]  
        state.append(state_max)
 
    return list(zip(words, state))

#### Evaluating tagging accuracy

In [55]:
# tagging the test sentences with modified_viterbi
start = time.time()
test_tagged_seq_using_modified_Viterbi = Modified_Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(test_tagged_seq_using_modified_Viterbi)

Time taken in seconds:  25.14567542076111
[('In', 'ADP'), ('major', 'ADJ'), ('market', 'NOUN'), ('activity', 'NOUN'), (':', '.'), ('On', 'ADP'), ('London', 'NOUN'), ("'s", 'PRT'), ('Stock', 'NOUN'), ('Exchange', 'NOUN'), (',', '.'), ('Reuters', 'NOUN'), ('shares', 'NOUN'), ('rose', 'VERB'), ('five', 'NUM'), ('pence', 'NOUN'), ('to', 'PRT'), ('913', 'NUM'), ('pence', 'NOUN'), ('-LRB-', '.'), ('$', '.'), ('14.43', 'NUM'), ('*U*', 'X'), ('-RRB-', '.'), ('.', '.'), ('The', 'DET'), ('court', 'NOUN'), ('hearing', 'NOUN'), ('began', 'VERB'), ('in', 'ADP'), ('early', 'ADJ'), ('October', 'NOUN'), ('at', 'ADP'), ('the', 'DET'), ('request', 'NOUN'), ('of', 'ADP'), ('Anthony', 'NOUN'), ('Hazell', 'NOUN'), (',', '.'), ('district', 'NOUN'), ('auditor', 'NOUN'), ('for', 'ADP'), ('Hammersmith', 'NOUN'), (',', '.'), ('who', 'PRON'), ('*T*-63', 'X'), ('argued', 'VERB'), ('that', 'ADP'), ('local', 'ADJ'), ('councils', 'NOUN'), ('are', 'VERB'), ("n't", 'ADV'), ('vested', 'VERB'), ('with', 'ADP'), ('consti

In [56]:
# accuracy
check = [i for i, j in zip(test_tagged_seq_using_modified_Viterbi, test_run_base) if i == j] 
accuracy_modified_Viterbi = len(check)/len(test_tagged_seq_using_modified_Viterbi)
accuracy_modified_Viterbi

print("Modified Viterbi Algorithm's Accuracy :",accuracy_modified_Viterbi)

Modified Viterbi Algorithm's Accuracy : 0.9927536231884058


In [58]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(test_tagged_seq_using_modified_Viterbi, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('and', 'CONJ'), (('punish', 'NOUN'), ('punish', 'VERB'))]]

In [59]:
## Testing on Vanilla Viterbi Algorithm on Test Sentences
sentence_test = 'Twitter is the best networking social site.Man is a social animal. Data science is an emerging field.Data science jobs are high in demand.'
# Android is a mobile operating system developed by Google.
words = word_tokenize(sentence_test)
start = time.time()
tagged_seq = Modified_Viterbi(words)
end = time.time()
difference = end-start
print(tagged_seq)

[('Twitter', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('best', 'ADJ'), ('networking', 'NOUN'), ('social', 'ADJ'), ('site.Man', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('social', 'ADJ'), ('animal', 'NOUN'), ('.', '.'), ('Data', 'NOUN'), ('science', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('emerging', 'VERB'), ('field.Data', 'NOUN'), ('science', 'NOUN'), ('jobs', 'NOUN'), ('are', 'VERB'), ('high', 'ADJ'), ('in', 'ADP'), ('demand', 'NOUN'), ('.', '.')]


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [60]:

print("Vanilla Viterbi Algorithm's Accuracy :",accuracy_vanilla_Viterbi)


Vanilla Viterbi Algorithm's Accuracy : 0.9202898550724637


In [61]:

print("Modified Viterbi Algorithm's Accuracy :",accuracy_modified_Viterbi)

Modified Viterbi Algorithm's Accuracy : 0.9927536231884058


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [63]:
1.Original-'Twitter', '.' Corrected - 'Twitter', 'NOUN'
2.Original Pos Tagger-'google', '.' Corrected- 'site', 'NOUN'
